In [ ]:
#필요 라이브러리 import
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

In [ ]:
water_df = pd.read_csv('../input/water-potability/water_potability.csv')
water_df

총 10개의 컬럼과 3276개의 row로 이루어진 데이터
* 각각의 컬럼에 대한 정보를 Kaggle에서 확인(링크 : https://www.kaggle.com/adityakadiwal/water-potability)
* Target 변수가 될 Potability 컬럼에서 0값은 마시지 못하는 물을 의미, 1값은 마실 수 있는 물을 의미

'Potability'컬럼 분포도 확인

In [ ]:
p_count = water_df['Potability'].value_counts()
print('Target 변수 count\n', p_count)
fig, ax = plt.subplots(figsize = (10, 8))
sns.countplot(x = 'Potability' , data = water_df, edgecolor = 'black')

분석을 위해 Target변수와 feature들을 분리
* 앞서 원본 데이터를 들여다 봤을때, NaN값을 포함하고 있는 피처들이 보이므로 이를 확인 -> info()
* 피처들의 분포도를 확인하여 정규화 or 표준화가 필요한지 확인 -> describe()

In [ ]:
target = water_df['Potability']
feature = water_df.drop(['Potability'], axis= 1)
feature.head(5)

In [ ]:
feature.info()

* 피처들의 dtype이 모두 숫자형 -> 따로 인코딩 적용이 필요하지 않음
* ph, Sulfate, Trihalomethanes 컬럼에 NaN값이 존재

결측치 비율 확인

In [ ]:
feature.isnull().sum()/len(feature)

XGBoost나 LightGBM같은 일부 알고리즘은 결측치를 잘 처리하지만 일반적인 알고리즘에서는 결측치에 의해 모델이 엉망이 될 수 있음
* RandomForest 알고리즘을 사용할 것이기 때문에 결측치 처리 필요
*  fillna메서드를 통해 가장 쉬운 Mean or Median Imputation을 사용하기 전에 혹시나 Categorical 변수가 있는 확인

In [ ]:
feature.describe()

Categorical 변수는 없으므로 평균값으로 NaN처리

In [ ]:
def fillna_mean(df, col):
    mean = df[col].mean()
    df[col] = df[col].fillna(mean)
    return df

feature = fillna_mean(feature, 'ph')
feature = fillna_mean(feature, 'Sulfate')
feature = fillna_mean(feature, 'Trihalomethanes')

print('전처리 후 NaN값 확인\n', feature.isnull().sum())
print('Feature Shape:', feature.shape)

In [ ]:
feature.describe()

In [ ]:
feature.head(5)

## Visualizaiton
* swarm plot을 이용해 시각화
* 원본 데이터 그대로 시각화하기엔 피처들끼리의 차이가 심하므로 표준화시켜서 plotting
* 표준화에는 사이킷런의 StandardScaler를 이용

In [ ]:
target = target.replace([0, 1], ['Unsafe', 'Safe'])

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(feature)
feature_scaled = pd.DataFrame(scaler.transform(feature), columns= feature.columns)

In [ ]:
data = pd.concat([target, feature_scaled], axis = 1)
data = pd.melt(data, id_vars= 'Potability', var_name= 'features', value_name= 'value')
fig, ax = plt.subplots(1, 1, figsize = (10, 10))
sns.violinplot(x = 'features', y = 'value', hue = 'Potability', data = data, split = True, inner = 'quart')
plt.xticks(rotation= 90)

In [ ]:
#Boxplot 그려보기
plt.figure(figsize = (10, 10))
sns.boxplot(x = 'features', y = 'value', hue = 'Potability', data = data)
plt.xticks(rotation = 45)

Plot들을 그려본 결과
* 각각의 피처들마다 Safe와 Unsafe의 차이가 별로 없음
* boxplot을 봤을때 이상치들이 보임

피처들간의 상관관계 알아보기 -> heatmap()

In [ ]:
fig, ax = plt.subplots(figsize = (15, 15))
sns.heatmap(feature.corr(), annot=True, fmt = '.2f')

상관계수가 높은 피처들이 존재하지 않으므로 피처들을 묶어서 모델링을 하는 작업이 필요하지 않는 것으로 보임

## Modeling
* RandomForest알고리과 하이퍼 파라미터 튜닝

RandomForest란?
* 배깅(Bagging)의 대표적인 알고리즘 
* 데이터가 중첩된 개별 데이터 세트에 결정 트리 분류기를 각각 적용하는 것
* 가장 큰 특징은 랜덤성(randomness)에 의해 트리들이 서로 조금씩 다른 특성을 갖는다는 점 -> 각 트리들의 예측들이 비상관화 -> 일반화 성능을 향상
* 랜덤화(Randomization)는 포레스트가 노이즈가 포함된 데이터에 대해서도 강인하게 만들어 줌

모델링을 적용하기 전, 머신러닝에서는 문자형 값을 허용하지 않으므로 시각화를 위해 문자형으로 바꿔준 target 변수를 다시 숫자형 값으로 변환

In [ ]:
target = target.replace(['Unsafe', 'Safe'], [0, 1])

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix
from sklearn.model_selection import cross_val_score

#학습 & 테스트 데이터 세트 분리
X_train, X_test, y_train, y_test = train_test_split(feature, target, test_size= 0.2, random_state= 0)

rf_clf = RandomForestClassifier(n_jobs= -1)
rf_clf.fit(X_train, y_train)
pred = rf_clf.predict(X_test)

In [ ]:
scores = cross_val_score(rf_clf, feature, target, cv = 5)
for iter_count, accuracy in enumerate(scores):
    print("교차 검증 {0} 정확도: {1:.4f}".format(iter_count, accuracy))

print('평균 정확도: {0:.4f}'.format(np.mean(scores))) 

In [ ]:
#평가 함수
def get_clf_eval(y_test, pred):
    confusion = confusion_matrix(y_test, pred)
    accuracy = accuracy_score(y_test, pred)
    precision = precision_score(y_test, pred)
    recall = recall_score(y_test, pred)
    print('오차 행렬')
    print(confusion)
    print('정확도: {0:.4f}, 정밀도:{1:.4f}, 재현율: {2:.4f}'.format(accuracy, precision, recall))

In [ ]:
get_clf_eval(y_test, pred)

### 하이퍼 파라미터 튜닝

트리 기반의 앙상블 알고리즘의 단점 : 하이퍼 파라미터가 너무 많고, 그로 인해서 튜닝을 위한 시간이 많이 소모. But, 튜닝 후 예측 성능이 크게 향상되는 경우가 많지 않음

랜덤 포레스트 하이퍼 파라미터
* n_estimators : 결정 트리의 개수를 지정, 많이 설정할수록 좋은 성능을 기대할 수 있지만 계속 증가시킨다고 무조건 향상X
* max_features : 최적의 분할을 위해 고려할 최대 피처 개수. 디폴트는 'auto' or 'sqrt'와 같음. 트리를 분할하는 피처를 참조할때 sqrt(전체 피처 개수)만큼 참조
* max_depth : 트리의 최대 깊이를 규정. 과적합을 제어
* min_samples_split : 노드를 분할하기 위한 최소한의 샘플 데이터 수. 과적합을 제어
* min_samples_leaf : 말단 노드(leaf)가 되기 위한 최소한의 샘플 데이터 수

In [ ]:
params = {
    'n_estimators' : [200],
    'max_depth' : [16, 18, 20, 22, 24],
    'max_features' : ['auto', 'sqrt', 'log2'],
    'min_samples_leaf' : [6, 8, 10],
    'min_samples_split' : [12, 14, 16, 18]
}

grid_cv = GridSearchCV(rf_clf, param_grid=params, cv = 5, n_jobs= -1)
grid_cv.fit(X_train, y_train)

print('최적 하이퍼 파라미터:\n', grid_cv.best_params_)
print('최고 예측 정확도: {0:.4f}'.format(grid_cv.best_score_))